In [85]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from matplotlib import pyplot as plt
import json
pd.options.display.max_colwidth = 300
import config
data_root= config.DATA_ROOT
from tqdm import tqdm
tqdm.pandas()
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np, os, sys
sys.path.append('helpers/')
from text_processing import preprocess_text
pd.options.display.max_colwidth=5000

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [99]:
remove_cols=['Unnamed: 0', 'all_awardings', 'allow_live_comments', 
       'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_type', 'author_fullname', 'author_patreon_flair',
       'author_premium',  'can_mod_post', 'contest_mode',
        'domain', 'full_link', 'gildings', 
       'is_crosspostable', 'is_meta', 'wls','removed_by_category', 'distinguished',
       'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
       'link_flair_background_color', 'link_flair_richtext','link_flair_template_id',
       'link_flair_text_color', 'link_flair_type', 'locked', 'media',
       'media_embed', 'media_only', 'parent_whitelist_status',
        'over_18', 'pinned', 'post_hint',
       'preview', 'retrieved_on', 'secure_media',
       'secure_media_embed', 'send_replies', 'spoiler', 'stickied',
        'subreddit_subscribers', 'subreddit_type',
       'thumbnail', 'thumbnail_height', 'thumbnail_width', 
        'treatment_tags',  'url','hidden', 'quarantine', 'subreddit_name_prefixed',
       'suggested_sort','og_description', 'og_title', 'brand_safe', 'rte_mode','previous_visits','can_gild','category','content_categories','hidden','quarantine','subreddit_name_prefixed','suggested_sort','approved_at_utc',#'view_count',
       'url_overridden_by_dest', 'author_is_blocked', 'is_created_from_ads_ui',
       'author_flair_background_color', 'author_flair_template_id',
       'author_flair_text_color', 'poll_data', 'gallery_data', 'is_gallery', 'banned_at_utc',
       'media_metadata', 'edited','no_follow', 'pwls', 'awarders', 'whitelist_status', 'banned_by','crosspost_parent_list','crosspost_parent', 'updated_utc','author_cakeday', 'gilded','updated_utc','steward_reports', 'removed_by', 'archived','link_flair_css_class','removal_reason',
]
remove_cols2=['Unnamed: 0',  
        'author_flair_background_color', 'author_flair_css_class',
       'author_flair_richtext', 'author_flair_template_id',
        'author_flair_text_color', 'author_flair_type',
        'author_patreon_flair', 'author_premium', 
       'banned_at_utc',  'can_mod_post', 'collapsed',
       'collapsed_because_crowd_control', 'collapsed_reason',
         'edited', 'gildings',
       'is_submitter', 'link_id', 'locked', 
       'permalink', 'retrieved_on',  'send_replies', 'stickied', 'top_awarded_type',
       'total_awards_received', 'treatment_tags']

## Load posts on subreddits

In [100]:
dfs = []
for n in ['student', 'education', 'teachers', 'professors', 'k12sysadmin', 'higheredsysadmin', 'edtechhelp']:
    df = pd.read_csv(data_root+'/reddit-posts/{}.csv'.format(n), sep=',', encoding='utf-8')
    dfs.append(df)
    print(n, len(df))
df = pd.concat(dfs)
df.drop(axis=1, columns=remove_cols, inplace=True)

df.shape

/var/folders/1p/0rwj4hf522b5v3bzw33bsdqw0000gn/T/ipykernel_23900/862881688.py:3: DtypeWarning: Columns (1,2,5,7,9,10,11,12,17,19,20,21,22,23,24,26,28,29,30,31,32,33,37,39,43,47,49,53,59,62,63,66,72,73,74,75,76,77,78,79,80,81,86,89,91,93,99,100) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_root+'/reddit-posts/{}.csv'.format(n), sep=',', encoding='utf-8')


student 8906


/var/folders/1p/0rwj4hf522b5v3bzw33bsdqw0000gn/T/ipykernel_23900/862881688.py:3: DtypeWarning: Columns (1,2,5,6,7,9,10,11,12,13,17,19,20,21,22,23,25,26,27,28,29,30,31,32,36,38,42,46,47,52,56,59,61,62,63,64,67,68,69,70,73,74,75,79,80,81,82,84,85,86,87,88,89,90,92,95,96,97,99,101,102) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_root+'/reddit-posts/{}.csv'.format(n), sep=',', encoding='utf-8')


education 334581


/var/folders/1p/0rwj4hf522b5v3bzw33bsdqw0000gn/T/ipykernel_23900/862881688.py:3: DtypeWarning: Columns (1,2,5,7,9,10,11,12,13,18,20,21,22,23,24,26,27,28,29,30,31,32,33,34,35,39,41,46,47,52,56,59,61,62,63,64,65,66,67,68,69,70,71,72,73,76,78,79,80,81,82,83,84,85,87,88,89,90,91,92,93,94,96,99,100,101,103,105) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_root+'/reddit-posts/{}.csv'.format(n), sep=',', encoding='utf-8')


...

/var/folders/1p/0rwj4hf522b5v3bzw33bsdqw0000gn/T/ipykernel_23900/862881688.py:3: DtypeWarning: Columns (2,12,19,24,26,27,29,32,34,35,39,40,51,87,88,89,90,91,92,93,94,95,97,100,102,104) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_root+'/reddit-posts/{}.csv'.format(n), sep=',', encoding='utf-8')


professors 23666


/var/folders/1p/0rwj4hf522b5v3bzw33bsdqw0000gn/T/ipykernel_23900/862881688.py:3: DtypeWarning: Columns (14,15,21,22,23,24,25,27,32,33,34,40,45,46,52,65,66,69,70,79,80,81,87,90,91,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_root+'/reddit-posts/{}.csv'.format(n), sep=',', encoding='utf-8')


k12sysadmin 12659
higheredsysadmin 97
edtechhelp 20


(545502, 20)

In [101]:
df['created_utc'] = pd.to_datetime(df.created_utc, unit='s')
df['author_created_utc']=pd.to_datetime(df.author_created_utc, unit='s')

In [14]:
def contains_keyword(text, keywords):
    if not isinstance(text, str):
        return False
    text =text.lower()
    for kw in keywords:
        if kw.lower() in text:
            return True
    return False
reddit_ed_kws = [line.split(',') for line in open('reddit-ed-kw.txt')][0]
reddit_priv_kws = [line.split(',') for line in open('reddit-priv-kw.txt')][0]
reddit_priv_kws

['privacy',
 ' security',
 ' surveillance',
 ' surveil',
 ' dataveillance',
 ' infosec',
 ' abuse',
 ' misuse']

In [102]:
ed_df=df[df.progress_apply(lambda row: contains_keyword(row.title, reddit_ed_kws) or \
                        contains_keyword(row.selftext, reddit_ed_kws), axis=1)]

ed_df.shape

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 545502/545502 [00:41<00:00, 13271.57it/s]


(7713, 20)

In [103]:
priv_df=ed_df[ed_df.progress_apply(lambda row: contains_keyword(row.title, reddit_priv_kws) or \
                        contains_keyword(row.selftext, reddit_priv_kws), axis=1)]
priv_df.set_index('id', inplace=True)
priv_df.shape

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7713/7713 [00:00<00:00, 22387.54it/s]


(189, 20)

In [104]:
priv_df.columns

Index(['author', 'created_utc', 'id', 'is_original_content', 'num_comments',
       'num_crossposts', 'permalink', 'score', 'selftext', 'subreddit',
       'subreddit_id', 'title', 'total_awards_received', 'upvote_ratio',
       'link_flair_text', 'author_id', 'author_created_utc', 'view_count',
       'mod_reports', 'user_reports'],
      dtype='object')

In [105]:
priv_df2=df[df.progress_apply(lambda row: contains_keyword(row.title, reddit_priv_kws) or \
                        contains_keyword(row.selftext, reddit_priv_kws), axis=1)]

priv_df2.shape

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 545502/545502 [00:17<00:00, 31454.57it/s]


(3792, 20)

In [44]:
priv_df.groupby('subreddit').selftext.count()

subreddit
Professors     45
Student         2
Teachers       81
education      10
k12sysadmin    43
Name: selftext, dtype: int64

In [203]:
priv_df.link_flair_text.unique()

array(['Policy &amp; Politics', 'Student',
       'Teacher Support &amp;/or Advice', 'Resignation', 'New Teacher',
       'Student Teacher Support &amp;/or Advice',
       'RANT [Students/Parents]', 'Career &amp; Interview Advice', nan,
       'Classroom Management &amp; Strategies', 'COVID-19',
       'Bad Teacher, No Apple', 'Pedagogy &amp; Best Practices',
       'SUCCESS!', 'Moderator Announcement', 'DISCUSSION :sloth:',
       'Rants / Vents', 'Advice / Support'], dtype=object)

In [43]:
priv_df.groupby('link_flair_text').selftext.count().sort_values(ascending=False)
# print(priv_df.groupby('link_flair_text').selftext.count().sort_values(ascending=False).to_frame().to_latex())

link_flair_text
Teacher Support &amp;/or Advice            21
Classroom Management &amp; Strategies       5
COVID-19                                    5
Student                                     4
Bad Teacher, No Apple                       4
New Teacher                                 4
Rants / Vents                               3
Advice / Support                            3
Pedagogy &amp; Best Practices               2
Policy &amp; Politics                       2
Resignation                                 2
Career &amp; Interview Advice               2
Student Teacher Support &amp;/or Advice     2
Curriculum &amp; Teaching Strategies        1
DISCUSSION :sloth:                          1
Ed Tech &amp; Tech Integration              1
Moderator Announcement                      1
Careers in Education                        1
RANT [Students/Parents]                     1
SUCCESS!                                    1
School Culture &amp; Policy                 1
Name: selftext, dt

### Analyze labelled posts

In [117]:
# labeled_df =pd.read_csv('priv-reddit-posts.csv')
# labeled_df.shape
# labeled_df = labeled_df[labeled_df.apply(lambda row: isinstance(row.Code, str), axis=1)]
# labeled_df.shape
labeled_df=pd.read_excel('priv-reddit-posts-labeled.xlsx')
labeled_df.set_index('id', inplace=True)
labeled_df.shape

(87, 20)

In [109]:
# labeled_df2 = priv_df.loc[labeled_df.index]
# labeled_df2['code'] = labeled_df.code
# labeled_df2.to_excel('priv-reddit-posts-labeled.xlsx')

In [32]:
labeled_df.groupby('subreddit').selftext.count().sort_values(ascending=False)

subreddit
Teachers       39
k12sysadmin    25
Professors     19
Name: selftext, dtype: int64

In [33]:
labeled_df.groupby('link_flair_text').selftext.count().sort_values(ascending=False)

link_flair_text
Teacher Support &amp;/or Advice            9
COVID-19                                   3
Classroom Management &amp; Strategies      3
Bad Teacher, No Apple                      2
Pedagogy &amp; Best Practices              2
Student                                    2
Advice / Support                           1
Policy &amp; Politics                      1
Resignation                                1
SUCCESS!                                   1
Student Teacher Support &amp;/or Advice    1
Name: selftext, dtype: int64

In [47]:
print(labeled_df.groupby(['code', 'subreddit']).selftext.count().to_frame().to_latex())#.head(10)

\begin{tabular}{llr}
\toprule
            &             &  selftext \\
code & subreddit &           \\
\midrule
edtech support & Professors &         2 \\
            & Teachers &         3 \\
edtech-generic & Professors &         1 \\
            & Teachers &         1 \\
edtech-ques & Professors &         3 \\
            & Teachers &        10 \\
            & k12sysadmin &         9 \\
other & Professors &         1 \\
            & Teachers &         4 \\
            & k12sysadmin &         1 \\
other-privacy & k12sysadmin &         3 \\
privacy incident & Professors &         1 \\
privacy-generic & Professors &         1 \\
            & Teachers &         2 \\
            & k12sysadmin &         1 \\
privacy-specific & Professors &         9 \\
            & Teachers &        18 \\
            & k12sysadmin &        10 \\
tech\_brandi & Professors &         1 \\
            & k12sysadmin &         1 \\
\bottomrule
\end{tabular}



/var/folders/1p/0rwj4hf522b5v3bzw33bsdqw0000gn/T/ipykernel_23900/3351673468.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(labeled_df.groupby(['code', 'subreddit']).selftext.count().to_frame().to_latex())#.head(10)


### TF-IDF words

In [86]:
def get_tfidf_top_features(documents,n_top=10):
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,stop_words='english')
    tfidf = tfidf_vectorizer.fit_transform(documents)
    importance = np.argsort(np.asarray(tfidf.sum(axis=0)).ravel())[::-1]
    tfidf_feature_names = np.array(tfidf_vectorizer.get_feature_names())
    return tfidf_feature_names[importance[:n_top]]


In [92]:
stopwords = set([line.strip() for line in open('stopwords-twitter-topic.txt')])
texts = [x for x in labeled_df[(labeled_df.code=='privacy-specific')].selftext.values if isinstance(x, str)]

texts = [' '.join(preprocess_text(text)) for text in texts]

corpus=list(texts)
top_words = (get_tfidf_top_features(corpus, n_top=25))
np.array(top_words)

array(['zoom', 'students', 'school', 'privacy', 'google', 'online',
       'goguardian', 'teacher', 'student', 'meeting', 'class', 'security',
       'policy', 'kids', 'camera', 'feel', 'proctoru', 'access',
       'teachers', 'issues', 'video', 'people', 'classroom', 'app',
       'parents'], dtype='<U13')

### Visibility

In [93]:
labeled_df.columns#groupby('code')

Index(['author', 'created_utc', 'is_original_content', 'num_comments',
       'num_crossposts', 'permalink', 'score', 'selftext', 'subreddit',
       'subreddit_id', 'title', 'total_awards_received', 'upvote_ratio',
       'link_flair_text', 'author_id', 'author_created_utc', 'mod_reports',
       'user_reports', 'code'],
      dtype='object')

In [114]:
labeled_df.created_utc.describe()

/var/folders/1p/0rwj4hf522b5v3bzw33bsdqw0000gn/T/ipykernel_23900/3306847447.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  labeled_df.created_utc.describe()


count                      87
unique                     87
top       2021-04-03 03:17:26
freq                        1
first     2014-11-17 23:37:06
last      2022-01-13 22:51:31
Name: created_utc, dtype: object

In [116]:
print(labeled_df[labeled_df.code=='privacy-specific'][['score',  'upvote_ratio', 'num_comments']].describe().to_latex())##.boxplot()

\begin{tabular}{lrrr}
\toprule
{} &      score &  upvote\_ratio &  num\_comments \\
\midrule
count &  40.000000 &     26.000000 &      40.00000 \\
mean  &   1.750000 &      0.995000 &      26.55000 \\
std   &   2.227451 &      0.023537 &      45.36233 \\
min   &   0.000000 &      0.880000 &       0.00000 \\
25\%   &   1.000000 &      1.000000 &       6.00000 \\
50\%   &   1.000000 &      1.000000 &      15.00000 \\
75\%   &   1.000000 &      1.000000 &      21.00000 \\
max   &  13.000000 &      1.000000 &     271.00000 \\
\bottomrule
\end{tabular}



/var/folders/1p/0rwj4hf522b5v3bzw33bsdqw0000gn/T/ipykernel_23900/2928238977.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(labeled_df[labeled_df.code=='privacy-specific'][['score',  'upvote_ratio', 'num_comments']].describe().to_latex())##.boxplot()
